In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd 빅데이터 입문

/content/drive/MyDrive/빅데이터 입문


In [5]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
movies = pd.read_csv('./movies.csv')
movies.head()

,tconst,titleType,isAdult,startYear,runtimeMinutes,genres_num,genres_first_encoded,first_director,first_writer,directors_count,writers_count,actor_count,actress_count,crew_total,first_actor,first_actress,averageRating,numVotes,score
0,tt0000009,0,0,1894,45,1,0,85156,85156,1,1,2,1,4,183823,63086,5.3,207,1
1,tt0000574,0,0,1906,70,3,1,846879,846879,1,1,3,1,10,846894,846887,6.0,853,0
2,tt0000591,0,0,1907,90,1,2,141150,141150,1,1,2,2,5,906197,1323543,5.0,21,1
3,tt0000615,0,0,1907,1995,1,2,533958,92809,1,2,5,1,9,3071427,218953,4.3,25,1
4,tt0000941,0,0,1909,45,1,2,63413,63413,2,3,3,1,7,34453,294022,4.5,27,1


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227877 entries, 0 to 227876
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                227877 non-null  object 
 1   titleType             227877 non-null  int64  
 2   isAdult               227877 non-null  int64  
 3   startYear             227877 non-null  int64  
 4   runtimeMinutes        227877 non-null  int64  
 5   genres_num            227877 non-null  int64  
 6   genres_first_encoded  227877 non-null  int64  
 7   first_director        227877 non-null  int64  
 8   first_writer          227877 non-null  int64  
 9   directors_count       227877 non-null  int64  
 10  writers_count         227877 non-null  int64  
 11  actor_count           227877 non-null  int64  
 12  actress_count         227877 non-null  int64  
 13  crew_total            227877 non-null  int64  
 14  first_actor           227877 non-null  int64  
 15  

In [8]:
X = movies.iloc[:, 1:16]
y = movies.iloc[:, 18]

# 특성 표준화
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state=321)

# 1) Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()

# parameter grid
parameters = {
    'penalty' : ['l1','l2'],
    'C'       : [0.001, 0.01, 0.1, 1, 10, 25, 50, 100],
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}

# gridsearch 객체
gridsearch_logistic = GridSearchCV(logistic,                  # model
                                   param_grid = parameters,   # hyperparameters
                                   scoring='accuracy',        # metric for scoring
                                   cv=2)                     # number of folds

# 그리드 서치 수행
best_model = gridsearch_logistic.fit(X_train, y_train)

In [11]:
result = pd.DataFrame(gridsearch_logistic.cv_results_['params'])
result['mean_test_score'] = gridsearch_logistic.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

,C,penalty,solver,mean_test_score
15,0.100,l2,newton-cg,0.567183
16,0.100,l2,lbfgs,0.567170
21,1.000,l2,newton-cg,0.567170
27,10.000,l2,newton-cg,0.567164
45,100.000,l2,newton-cg,0.567164
33,25.000,l2,newton-cg,0.567164
39,50.000,l2,newton-cg,0.567164
34,25.000,l2,lbfgs,0.567151
46,100.000,l2,lbfgs,0.567151
40,50.000,l2,lbfgs,0.567151


In [12]:
# 테스트 세트 점수
test_score = gridsearch_logistic.score(X_test, y_test)

print("테스트 세트 점수: {:.2f}".format( test_score ))
print("최적 매개변수: {}".format(gridsearch_logistic.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(gridsearch_logistic.best_score_))

테스트 세트 점수: 0.56
최적 매개변수: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
최고 교차 검증 점수: 0.57


In [13]:
logistic = LogisticRegression(C= 0.1, penalty= 'l2', solver= 'newton-cg')
logistic.fit(X_train,y_train)
pred = logistic.predict(X_test)
accuracy = accuracy_score(y_test,pred)
accuracy

0.5643906149435375

# 2) Random Forest

In [14]:
params ={
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

rf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_rf = GridSearchCV(rf, param_grid=params, cv=2,scoring='accuracy', n_jobs=-1)


best_model = grid_rf.fit(X_train,y_train)

In [15]:
# 테스트 세트 점수
test_score = grid_rf.score(X_test, y_test)

print("테스트 세트 점수: {:.2f}".format( test_score ))
print("최적 매개변수: {}".format(grid_rf.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(grid_rf.best_score_))

테스트 세트 점수: 0.64
최적 매개변수: {'max_depth': 12, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 100}
최고 교차 검증 점수: 0.64


In [16]:
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정

rf = RandomForestClassifier(max_depth= 12,
                            min_samples_leaf= 12,
                            min_samples_split=8,
                            n_estimators= 100,
                            class_weight="balanced",
                            random_state=0)
rf.fit(X_train,y_train)
pred = rf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
accuracy

0.5852641741267334

In [17]:
from sklearn.metrics import classification_report

print("RandomForest")
print(classification_report(y_test,pred))

RandomForest
              precision    recall  f1-score   support

           0       0.69      0.64      0.66     37885
           1       0.56      0.53      0.54     29212
           2       0.08      0.37      0.13      1267

    accuracy                           0.59     68364
   macro avg       0.44      0.51      0.45     68364
weighted avg       0.62      0.59      0.60     68364



# 3) AdaBoostClassifier

In [18]:
params ={
    'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1.0]
}

adaboost = AdaBoostClassifier(random_state=10)
grid_adaboost = GridSearchCV(adaboost, param_grid=params, cv=2, scoring='accuracy')

best_model = grid_adaboost.fit(X_train,y_train)

In [19]:
# 테스트 세트 점수
test_score = grid_adaboost.score(X_test, y_test)

print("테스트 세트 점수: {:.2f}".format( test_score ))
print("최적 매개변수: {}".format(grid_adaboost.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(grid_adaboost.best_score_))

테스트 세트 점수: 0.62
최적 매개변수: {'learning_rate': 1.0, 'n_estimators': 100}
최고 교차 검증 점수: 0.62


In [20]:
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
adaboost = AdaBoostClassifier(learning_rate = 1.0, n_estimators = 90, random_state=10)
adaboost.fit(X_train,y_train)
pred = adaboost.predict(X_test)
accuracy = accuracy_score(y_test,pred)
accuracy

0.6194634603007431

In [21]:
from sklearn.metrics import classification_report

print("AdaBoostClassifier")
print(classification_report(y_test,pred))

AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.64      0.77      0.70     37885
           1       0.58      0.45      0.51     29212
           2       0.00      0.00      0.00      1267

    accuracy                           0.62     68364
   macro avg       0.41      0.41      0.40     68364
weighted avg       0.60      0.62      0.60     68364



# 4) BaggingClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

params = {"base_estimator__max_depth": [3,5,10,20],
          "base_estimator__min_samples_leaf": [1, 3, 5, 7, 10],
          "base_estimator__min_samples_split": [2, 5, 7],
          'bootstrap_features': [False, True],
          'max_features': [0.5, 0.7, 1.0],
          'max_samples': [0.5, 0.7, 1.0],
          'n_estimators': [2, 5, 10, 20],
}

bgcl = BaggingClassifier(DecisionTreeClassifier())
grid_bgcl = GridSearchCV(bgcl, param_grid=params, cv=2, scoring='accuracy')

best_model = grid_bgcl.fit(X_train,y_train)

In [ ]:
# 테스트 세트 점수
test_score = grid_bgcl.score(X_test, y_test)

print("테스트 세트 점수: {:.2f}".format( test_score ))
print("최적 매개변수: {}".format(grid_bgcl.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(grid_bgcl.best_score_))

In [ ]:
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
bgcl = BaggingClassifier(DecisionTreeClassifier(), learning_rate = 1.0, n_estimators = 90, random_state=10)
bgcl.fit(X_train,y_train)
pred = bgcl.predict(X_test)
accuracy = accuracy_score(y_test,pred)
accuracy

# 5) GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

gbcl = GradientBoostingClassifier()
grid_gbcl = GridSearchCV(gbcl, param_grid=params, cv=2, scoring='accuracy')

best_model = grid_gbcl.fit(X_train,y_train)

In [ ]:
# 테스트 세트 점수
test_score = grid_gbcl.score(X_test, y_test)

print("테스트 세트 점수: {:.2f}".format( test_score ))
print("최적 매개변수: {}".format(grid_gbcl.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(grid_gbcl.best_score_))

In [ ]:
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
gbcl = GradientBoostingClassifier()
gbcl.fit(X_train,y_train)
pred = gbcl.predict(X_test)
accuracy = accuracy_score(y_test,pred)
accuracy

# 6) XGBClassifier

In [ ]:
from xgboost import XGBClassifier

params={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['binary:logistic'],
                 'random_state':[2]}

xgb = XGBClassifier()
grid_xgb = GridSearchCV(xgb, param_grid=params, cv=2, scoring='accuracy')

best_model = grid_xgb.fit(X_train,y_train)

In [ ]:
# 테스트 세트 점수
test_score = grid_xgb.score(X_test, y_test)

print("테스트 세트 점수: {:.2f}".format( test_score ))
print("최적 매개변수: {}".format(grid_xgb.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(grid_xgb.best_score_))

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, np.ravel(y_train,order='C'))

pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, pred)
accuracy